In [5]:
from scipy import stats
import numpy as np
import pandas as pd

# Конверсия

Общий алгоритм:

1. Определяем размер выборки
2. Запускаем тест и останавливаем его в момент достаточного числа пользователей
3. Считаем доверительные интервалы для разницы и p-value для z-тестов для разницы конверсий
4. Если вариантов больше 2, то учитываем поправку FDR
5. Делаем вывод о победителях или вывод, что победитель однозначно не определен
6. Если победитель не определен, то можно запустить повторный тест с 5 шагами, описанными выше.

## 1. Определяем размер выборки

$
n = \frac{(z_{\alpha/2} + z_{\beta})^2} {(p_1 - p_2)^2} * p_1 * (1 - p_1) + p_2 * (1-p_2)
$

где $p_1$ - базовая конверсия, $p_2$ - конверсия при предполагаемом эффекте
$z_{\alpha}$ - $\alpha$ квантиль нормального распределения
$z_{\beta}$ - $\beta$ квантиль нормального распределения

In [9]:
# Пример
def get_size_zratio(p_control_percent, mde_percent, alpha=0.05, beta=0.2,
                    type="equality"):
    """
    :param p_control_percent: conversion rate in percent (10 means 10% conversion)
    :param mde_percent: mde in percent
    :param alpha: error I
    :param beta: error II
    :param type: hypothesis
    :return:
    """
    z_alpha = stats.norm.ppf(1 - alpha/2)
    z_beta = stats.norm.ppf(1 - beta)
    # Rewrite to notice book 2007 year (SAMPLE SIZE CALCULATION FOR COMPARING PROPORTIONS)
    p1, mde_test = p_control_percent / 100, -(p_control_percent * mde_percent) / 10000
    p2 = p1 - mde_test
    n = ((z_alpha + z_beta) ** 2 / (p1 - p2) ** 2) * (p1 * (1 - p1) + p2 * (1 - p2))
    return np.uint32(n)
p_control, mde_percent = 11, 10
print(f"Для базовой конверсии в {p_control}% и ожидаемом минимальном эффекте \n"
      f"в {mde_percent}% необходимое ч  исло наблюдений для каждого варианта: \n"
      f"{get_size_zratio(p_control, mde_percent)}")

Для базовой конверсии в 11% и ожидаемом минимальном эффекте 
в 10% необходимое ч  исло наблюдений для каждого варианта: 
13249
